
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# Planning a Compound AI System Architecture

In this demo, we will plan a compound AI system architecture using pure python. The goal is to define the scope, functionalities and constraints of the system to be developed. 

We will create the system architecture to outline the structure and relationship of each component of the system. At this stage, we need to address the technical challenges and constraints of language model and frameworks to be used. 

**Learning Objectives:**

*By the end of this demo, you will be able to*:

* Apply a class architecture to the stages identified during Decomposition

* Explain a convention that maps stage(s) to class methods

* Plan what method attributes to use when writing a compound application

* Identify various components in a compound app


## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **14.3.x-cpu-ml-scala2.12 14.3.x-scala2.12**



## Classroom Setup

Before starting the demo, **run the following code cells**.

In [0]:
%sh
apt-get install -y graphviz

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  fonts-liberation libann0 libcdt5 libcgraph6 libgts-0.7-5 libgts-bin libgvc6
  libgvpr2 liblab-gamut1 libpathplan4
Suggested packages:
  gsfonts graphviz-doc
The following NEW packages will be installed:
  fonts-liberation graphviz libann0 libcdt5 libcgraph6 libgts-0.7-5 libgts-bin
  libgvc6 libgvpr2 liblab-gamut1 libpathplan4
0 upgraded, 11 newly installed, 0 to remove and 41 not upgraded.
Need to get 4680 kB of archives.
After this operation, 10.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-liberation all 1:1.07.4-11 [822 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libann0 amd64 1.1.2+doc-7build1 [26.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libcdt5 amd64 2.42.2-6ubuntu0.1 [21.1 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/unive

debconf: delaying package configuration, since apt-utils is not installed


Fetched 4680 kB in 2s (2673 kB/s)
Selecting previously unselected package fonts-liberation.
(Reading database ... 106420 files and directories currently installed.)
Preparing to unpack .../00-fonts-liberation_1%3a1.07.4-11_all.deb ...
Unpacking fonts-liberation (1:1.07.4-11) ...
Selecting previously unselected package libann0.
Preparing to unpack .../01-libann0_1.1.2+doc-7build1_amd64.deb ...
Unpacking libann0 (1.1.2+doc-7build1) ...
Selecting previously unselected package libcdt5:amd64.
Preparing to unpack .../02-libcdt5_2.42.2-6ubuntu0.1_amd64.deb ...
Unpacking libcdt5:amd64 (2.42.2-6ubuntu0.1) ...
Selecting previously unselected package libcgraph6:amd64.
Preparing to unpack .../03-libcgraph6_2.42.2-6ubuntu0.1_amd64.deb ...
Unpacking libcgraph6:amd64 (2.42.2-6ubuntu0.1) ...
Selecting previously unselected package libgts-0.7-5:amd64.
Preparing to unpack .../04-libgts-0.7-5_0.7.6+darcs121130-5_amd64.deb ...
Unpacking libgts-0.7-5:amd64 (0.7.6+darcs121130-5) ...
Selecting previously uns

In [0]:
%pip install -U --quiet graphviz

dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup-01

## Overview of application

In  notebook 1.1 - Multi-stage Deconstruct we created the sketch of our application below. Now it's time to fill in some of the details about each stage. Approach is more art than science, so this activity, we'll set convention for our planning that we want to define the following method attributes for each of our stages:
 * **Intent**: Provided from our previous exercise. Keep this around, when you get into actual coding this will be the description part of your docstring, see [PEP-257
](https://peps.python.org/pep-0257/).
 * **Name**: YES! Naming things is hard. You'll get a pass in the exercise because we provide the name for you keep the content organized, but consider how you would have named things. Would you have used the `run_` prefix? Also remember that [PEP-8](https://peps.python.org/pep-0008/#method-names-and-instance-variables) already provides some conventions, specifically:
     * lowercase with words separated by underscores as necessary to improve readability
     * Use one leading underscore only for non-public methods and instance variables (not applicable to our exercise here)
     * Avoid name clashes with subclasses
 * **Dependencies**: When planning you'll likely already have an idea of approach or library that you'll need in each stage. Here, you will want to capture those dependencies. After looking at those dependencies you may notice that you'll need more     
 * **Signature**: These are the argument names and types as well as the output type. However, when working with compound apps it's helpful to have stage methods that are directly tied to an LLM type of chat or completion to take the form:
     * **model_inputs**: These are inputs that will change with each request and are not a configuration setting in the application.
     * **params**: These are additional arguments we want exposed in our methods, but will likely not be argumented by users once the model is in model serving.
     * **output**: This is the output of a method and will commonly take the form of the request response of a served model if one is called within the method.


 **NOTE**: At this point in planning, you don't necessarily need to get into the decisions about what arguments should be a compound app class entity and which should be maintained as class members.

 **NOTE**: The separation of model_inputs and params is an important one. Compound applications accumulate a lot of parameters that will need to have defaults set during class instantion or load_context calls. By separating those arguments in the planning phase, it will be easier to identify the parameter space that is configurable in your compound application. While not exactly the same, it may be helpful to think of this collection of parameters as hyperparameters - these are configurations will spend time optimizing prior to best application selection, but not set during inference.


In [0]:
displayHTML(html_run_search_1.replace("[SEARCH_GRAPHIC]", get_stage_html('search')))

Attribute,Considerations,Student Answer,Instruction Approach
Name,"Name the method, be succinct",,run_search
Dependencies,"If we use a Databricks VectorSearchClient with a VectorSearchIndex, which methods would we use?",,databricks.vector_search.index.VectorSearchIndex.get_indexdatabricks.vector_search.index.VectorSearchIndex.similarity_search
Application-Arguments,What configurations for this stage would we want to set as an application configuration?,,We'll want to have the vector search index set during instantiation. The two arguments we'll need for that are:endpoint_name: strindex_name: str
Signature-Input,What input will we provide to our search?,,We'll want to provide the question being asked to search against. To provide our question as text similarity_search uses:query_text: str
Signature-Params,What parameters can be provided to our search?,,Anything that isn't our input could be a parameter. For similarity_search those are:columns: [str]filters: strnum_results: intdebug_level: str
Signature-Output,What kind of output should we define for this stage? The output of similarity_search string of dict. Add some structure and define the output as a dataclass.,,Similarity_search returns a dict that includes all n search results. In this MVP we'll define that result as a dataclass based on the response structure to simplify handling in our next stage run_augment:@dataclassclass SimilaritySearchResult: manifest: dict = ... result: dict = ... next_page_token: str debug_info: dict = ...


In [0]:
displayHTML(html_run_search_2.replace("[SUMMARY_GRAPHIC]", get_stage_html('summary')))

Attribute,Considerations,Student Answer,Instruction Approach
Name,"Name the method, be succinct",,run_summary
Dependencies,If we use a DatabricksDeploymentClient to run a completion llm. Which methods would we use?,,mlflow.deployments.get_deploy_clientmlflow.deployments.DatabricksDeploymentClient.predict
Application-Arguments,What configurations for this stage coreoutine would we want to set as an application configuration? Assume that we want to use the same model endpoint for all summary predicts within the application.,,"We'll want to have the deploy_client set during instantiation. Since we know the deploy client will be Databricks, we can instantiate with a static argument, get_deploy_client(""databricks"") To keep the model_endpoint consistant across call, we'll make the model_endpoint used for summary provided as an application argument. Thus, the predict method will have one argument populated from an application argument:endpoint_name: str"
Signature-Input,What input will we provide to the completion model to get a summary and relavance score? Assume you already know that the required input is a prompt. What two variables should the prompt template take?,,"From Text Completion Docs we see that we need to provide a prompt. We'll want to use both the content we are summarizing as well as the original question, thus:content: strquestion: str"
Signature-Params,What parameters can be provided to our summary model? Assume that the model type we are using is a completion model. Refer to the inputs from Text Completion Docs,,"Anything that isn't prompt could be a parameter. For a completion LLM type we could parameterize:max_tokens: inttemperature: floatstop: [str]n: intstream: boolextra_params: dictAbove, we assumed the use of a prompt template. Thus another parameter that we'll have for our stage coroutine is that prompt template:summary_prompt: str"
Signature-Output,What kind of output should we plan for this stage coroutine?,,"Our output from predict will be a response dict. From that, we'll want to extract the summary and relevance score. For debugging purposes, we should also include the id and content from the original search and define as a dataclass:@dataclassclass SearchResultAugmentedContent: id: int content: str summerization: str relevanceScore: float"


In [0]:
displayHTML(html_run_search_3.replace("[AUGMENT_GRAPHIC]", get_stage_html('augment')))

Attribute,Considerations,Student Answer,Instruction Approach
Name,"Name the method, be succinct",,run_augment
Dependencies,"Use asyncio to execute run_summary coroutines asynchronously. You will need to have an event loop, make the run_summary method execute as a coroutine, and gather the results.",,asyncio.get_event_loopasyncio.to_threadasyncio.gather
Signature-Input,"What input will we provide to run_augment? Consider what output we have from the prior stage, run_search.",,Same as output from run_search: SimilaritySearchResult
Signature-Output,What kind of output will we define for this stage? The gather method from asyncio creates a list. Thus our output should be a list of run_summary output.,,[SearchResultAugmentedContent]


In [0]:
displayHTML(html_run_search_4.replace("[GET_CONTEXT_GRAPHIC]", get_stage_html('get_context')))

Attribute,Considerations,Student Answer,Instruction Approach
Name,"Name the method, be succinct",,run_get_context
Dependencies,We have a list and we need to sort it. We can do this python pure.,,list.sort
Signature-Input,"What input will we provide to run_get_context? Consider what output we have from the prior stage, run_augment.",,Same as output from run_augment: [SearchResultAugmentedContent]
Signature-Output,What kind of output will we define for this stage? Intent is to simply consolidate the top three results into a single string and use that as context in the QA model.,,context: str


In [0]:
displayHTML(html_run_search_5.replace("[QA_GRAPHIC]", get_stage_html('qa')))

Attribute,Considerations,Student Answer,Instruction Approach
Name,"Name the method, be succinct",,run_qa
Dependencies,If we use a DatabricksDeploymentClient to run a chat llm. Which methods would we use? Hint: we can use the same dependencies for completion llms as we will use for chat llms.,,mlflow.deployments.get_deploy_clientmlflow.deployments.DatabricksDeploymentClient.predict
Application-Arguments,What configurations for this stage would we want to set as an application configuration? Assume that we want to use the same model endpoint for all QA predicts within the application.,,"We'll want to have the deploy_client set during instantiation. Since we know the deploy client will be Databricks, we can instantiate with a static argument, get_deploy_client(""databricks"") To keep the model_endpoint consistant across calls, we'll make the model_endpoint used for QA provided as an application argument. Thus, the predict method will have one argument populated from an application argument:endpoint_name: str"
Signature-Input,What input will we provide to the chat model the uses our context and the original question? Assume that we will again use a prompt template. What two variables should the prompt template take?,,"From Chat Model Docs we see that we need to provide messages. The messages format is a list of dict to handle message history, but we'll just need to coerse a prompt into this format. Our prompt will take the following:context: strquestion: str"
Signature-Params,What parameters can be provided to our qa model? Assume that the model type we are using is a chat model. Refer to the inputs from Chat Model Docs,,"Anything that isn't prompt could be a parameter. For a chat LLM type we could parameterize:max_tokens: inttemperature: floatAbove, we assumed the use of a prompt template. Thus another parameter that we'll have for qa is our own:qa_prompt: str"
Signature-Output,"What kind of output should we plan for this stage? The final output of the model is an answer and single string. However, we would like the full response available as an output of this stage.",,"Our output from predict will be a response dict. From that, we'll put that in a dataclass for ease of use:@dataclassclass SummaryModelResult: id: int object: str model: str choices: [dict] = ... usage: dict = ..."


In [0]:
displayHTML(html_run_search_6.replace("[MAIN_GRAPHIC]", get_stage_html('main')))

Attribute,Considerations,Student Answer,Instruction Approach
Name,"Name the method, be succinct",,main
Signature-Input,"Using the code above, what is our input?",,question: str
Signature-Output,"Using the code above, what is our output?",,str


In [0]:
displayHTML(html_run_search_7.replace("[MAIN_GRAPHIC]", get_stage_html('main')))


## Full Multi-Endpoint Architecture

We've gone through all the work of identifying the dependencies which include both a Data Serving Endpoing and a couple model serving endpoints. We should have a look at what our final architecture is. Even in this straight forward compound application, you can see that it has a lot of endpoint dependencies. It's worth having this perspective to see all the serving endpoints that must be maintained.

In [0]:
displayHTML(get_multistage_html())

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: pt Pages: 1 -->
 
 
 pt 
 
 
 cluster_workflow 
 
 Model Serving 
 
 
 cluster_app 
 
 Compound_rag_app Class 
 
 
 cluster_main 
 
 main() 
 
 
 cluster_run_augment 
 
 run_augment() 
 
 
 cluster_summary 
 
 Model Serving 
 
 
 cluster_vectorsearch 
 
 Data Serving 
 
 
 cluster_qa 
 
 Model Serving 
 
<!-- question -->
 
 question 
 
 question 
 
<!-- run_search -->
 
 run_search 
 
 run_search() 
 
<!-- question->run_search -->
 
 question->run_search 
 
 
 
<!-- run_summary -->
 
 run_summary 
 
 run_summary() 
 
<!-- run_search->run_summary -->
 
 run_search->run_summary 
 
 
 
<!-- vector_search -->
 
 vector_search 
 
 Databricks 
 VectorSearch 
 
<!-- run_search->vector_search -->
 
 run_search->vector_search 
 
 
 
<!-- run_get_context -->
 
 run_get_context 
 
 run_get_context() 
 
<!-- run_summary->run_get_context -->
 
 run_summary->run_get_context 
 
 
 
<!-- summary_llm -->
 
 summary_llm 
 
 Summary LLM 
 
<!-- run_summary->summary_llm -->
 
 run_summary->summary_llm 
 
 
 
<!-- run_qa -->
 
 run_qa 
 
 run_qa() 
 
<!-- run_get_context->run_qa -->
 
 run_get_context->run_qa 
 
 
 
<!-- qa_llm -->
 
 qa_llm 
 
 QA LLM 
 
<!-- run_get_context->qa_llm -->
<!-- answer -->
 
 answer 
 
 answer 
 
<!-- run_qa->answer -->
 
 run_qa->answer 
 
 
 
<!-- run_qa->qa_llm -->
 
 run_qa->qa_llm 
 
 
 
<!-- vector_search->run_search -->
 
 vector_search->run_search 
 
 
 
<!-- vector_search->summary_llm -->
<!-- hidden_search_offset -->
<!-- hidden_search_offset->run_search -->
<!-- hidden_search_offset->vector_search -->
<!-- summary_llm->run_summary -->
 
 summary_llm->run_summary 
 
 
 
<!-- qa_llm->run_qa -->
 
 qa_llm->run_qa


## Conclusion

In this demo, we planned a sample compound AI system using pure code. This demo showed how different components can be defined independently and then are linked together to build the system.


&copy; 2024 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>